**Sample ID**: 127

**Query**:

Can you restore some blog posts for me?

**DB Type**: Base Case

**Case Description**:

The user wants to restore blog posts that have been moved to the trash in their Confluence 'BLOG' space. There are two blog posts, "Annual Tech Summit Recap"  and "Q3 Marketing Goals" both of which currently have a "trashed" status. The user will confirm that both should be restored.

```
<multiturn info>
Space Name: They're in the 'BLOG' space. (Information Gathering)
Confirmation: "Annual Tech Summit Recap" and "Q3 Marketing Goals".(Information Gathering)
</multiturn info>
```

**Global/Context Variables:**


**APIs:**

- confluence


# Set Up

## Download relevant files

In [20]:
import io
import os
import sys
import zipfile
import shutil
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# Version to download
VERSION = "0.1.4"  # This will be replaced dynamically

# Define paths
CONTENT_DIR = '/content'
APIS_DIR = os.path.join(CONTENT_DIR, 'APIs')
DBS_DIR = os.path.join(CONTENT_DIR, 'DBs')
SCRIPTS_DIR = os.path.join(CONTENT_DIR, 'Scripts')
FC_DIR = os.path.join(CONTENT_DIR, 'Schemas')
ZIP_PATH = os.path.join(CONTENT_DIR, f'APIs_V{VERSION}.zip')

# Google Drive Folder ID where versioned APIs zip files are stored
APIS_FOLDER_ID = '1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4'

# List of items to extract from the zip file
ITEMS_TO_EXTRACT = ['APIs/', 'DBs/', 'Scripts/', 'Schemas/']

# Clean up existing directories and files
for path in [APIS_DIR, DBS_DIR, SCRIPTS_DIR, FC_DIR, ZIP_PATH]:
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)

# Authenticate and create the drive service
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Helper function to download a file from Google Drive
def download_drive_file(service, file_id, output_path, file_name=None, show_progress=True):
    """Downloads a file from Google Drive"""
    destination = output_path
    request = service.files().get_media(fileId=file_id)
    with io.FileIO(destination, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            if show_progress:
                print(f"Download progress: {int(status.progress() * 100)}%")

# 1. List files in the specified APIs folder
print(f"Searching for APIs zip file with version {VERSION} in folder: {APIS_FOLDER_ID}...")
apis_file_id = None

try:
    query = f"'{APIS_FOLDER_ID}' in parents and trashed=false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])
    for file in files:
        file_name = file.get('name', '')
        if file_name.lower() == f'apis_v{VERSION.lower()}.zip':
            apis_file_id = file.get('id')
            print(f"Found matching file: {file_name} (ID: {apis_file_id})")
            break

except Exception as e:
    print(f"An error occurred while listing files in Google Drive: {e}")

if not apis_file_id:
    print(f"Error: Could not find APIs zip file with version {VERSION} in the specified folder.")
    sys.exit("Required APIs zip file not found.")

# 2. Download the found APIs zip file
print(f"Downloading APIs zip file with ID: {apis_file_id}...")
download_drive_file(drive_service, apis_file_id, ZIP_PATH, file_name=f'APIs_V{VERSION}.zip')

# 3. Extract specific items from the zip file to /content
print(f"Extracting specific items from {ZIP_PATH} to {CONTENT_DIR}...")
try:
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_contents = zip_ref.namelist()

        for member in zip_contents:
            extracted = False
            for item_prefix in ITEMS_TO_EXTRACT:
                if member == item_prefix or member.startswith(item_prefix):
                    zip_ref.extract(member, CONTENT_DIR)
                    extracted = True
                    break

except zipfile.BadZipFile:
    print(f"Error: The downloaded file at {ZIP_PATH} is not a valid zip file.")
    sys.exit("Invalid zip file downloaded.")
except Exception as e:
    print(f"An error occurred during extraction: {e}")
    sys.exit("Extraction failed.")

# 4. Clean up
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

# 5. Add APIs to path
if os.path.exists(APIS_DIR):
    sys.path.append(APIS_DIR)
else:
    print(f"Error: APIS directory not found at {APIS_DIR} after extraction. Cannot add to path.")

# 6. Quick verification
# Check for the presence of the extracted items
verification_paths = [APIS_DIR, DBS_DIR, SCRIPTS_DIR]
all_present = True
print("\nVerifying extracted items:")
for path in verification_paths:
    if os.path.exists(path):
        print(f"✅ {path} is present.")
    else:
        print(f"❌ {path} is MISSING!")
        all_present = False

if all_present:
    print(f"\n✅ Setup complete! Required items extracted to {CONTENT_DIR}.")
else:
    print("\n❌ Setup failed! Not all required items were extracted.")
os.chdir(CONTENT_DIR)

Searching for APIs zip file with version 0.1.4 in folder: 1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4...
Found matching file: APIs_V0.1.4.zip (ID: 1TnAaWGfVrMxWTilyhy46-Aue_bh0XkNk)
Download progress: 100%
Extracting specific items from /content/APIs_V0.1.4.zip to /content...

Verifying extracted items:
✅ /content/APIs is present.
✅ /content/DBs is present.
✅ /content/Scripts is present.

✅ Setup complete! Required items extracted to /content.


## Install Dependencies and Clone Repositories

In [21]:
!pip install -r /content/APIs/requirements.txt

## Import APIs and initiate DBs

In [22]:
# proto_ignore
import random
import sys
import uuid

def patch_uuid4_deterministic(seed=42):
    rng = random.Random(seed)

    def deterministic_uuid4():
        return uuid.UUID(int=rng.getrandbits(128))

    sys.modules['uuid'].uuid4 = deterministic_uuid4

patch_uuid4_deterministic()

In [23]:
import confluence

confluence.SimulationEngine.db.load_state("/content/DBs/ConfluenceDefaultDB.json")

# Initialize Global variables
space = "BLOG"
post_1="Annual Tech Summit Recap"
post_2="Q3 Marketing Goals"

# Create two blog posts in the same blog space

# Post 1: Annual Tech Summit Recap
tech_summit_post = confluence.create_content({
    "type": "blogpost",
    "spaceKey": space,
    "title": post_1,
    "body": {
        "storage": {
            "value": "The Annual Tech Summit featured discussions on AI, quantum computing, and next-gen networking. Keynote sessions highlighted industry shifts and future strategies.",
            "representation": "storage"
        }
    }
})

# Post 2: Q3 Marketing Goals
q3_goals_post = confluence.create_content({
    "type": "blogpost",
    "spaceKey": space,
    "title": post_2,
    "body": {
        "storage": {
            "value": "Our Q3 marketing goals include expanding outreach in Europe, boosting engagement through webinars, and launching a new social media campaign for product awareness.",
            "representation": "storage"
        }
    }
})

confluence.delete_content(tech_summit_post["id"])
confluence.delete_content(q3_goals_post["id"])

# Initial Assertion

1. Assert that "BLOG" space exists.
2. Assert that there are exactly 2 blog posts in the "BLOG" space.
3. Assert that blog posts, "Annual Tech Summit Recap"  and "Q3 Marketing Goals" have the status "trashed".

In [24]:
from Scripts.assertions_utils import *
import confluence

# --- Constants ---
space = "BLOG"
post_1 = "Annual Tech Summit Recap"
post_2 = "Q3 Marketing Goals"

# --- Assertion 1: Assert that the Confluence space "BLOG" exists ---
api_error_1 = None
all_spaces = []

try:
    all_spaces = confluence.get_spaces()
except Exception as e:
    api_error_1 = str(e)

# Use compare_is_list_subset utility for membership check
assert_condition_1 = (
    api_error_1 is None and
    compare_is_list_subset(space, [s.get("spaceKey") for s in all_spaces])
)
assert_message_1 = (
    f"Assertion 1 Failed: Error occurred while fetching spaces: {api_error_1}"
    if api_error_1 else
    f"Assertion 1 Failed: Expected a Confluence space with key '{space}', but none was found."
)
assert assert_condition_1, assert_message_1


# --- Assertion 2: Assert that there are exactly 2 blog posts in the "BLOG" space ---
api_error_2 = None
blog_posts = []

try:
    blog_space = confluence.get_content_list(spaceKey=space, status="any")
    blog_posts = [
        post for post in blog_space
        if compare_strings(post.get("type"), "blogpost")
    ]
except Exception as e:
    api_error_2 = str(e)

assert_condition_2 = api_error_2 is None and len(blog_posts) >= 2
# Updated message: more accurate reflection of count expectation
assert_message_2 = (
    f"Assertion 2 Failed: Error occurred while fetching content for space '{space}': {api_error_2}"
    if api_error_2 else
    f"Assertion 2 Failed: Expected at least 2 blog posts in the '{space}' space, but found {len(blog_posts)}."
)
assert assert_condition_2, assert_message_2


# --- Assertion 3: Assert that both specified blog posts are present in the "trashed" state in the space ---
api_error_3 = None
trashed_titles = set()

try:
    trashed_blogposts = confluence.get_content_list(spaceKey=space, status="trashed")
    trashed_blogposts = [
        post for post in trashed_blogposts
        if compare_strings(post.get("type"), "blogpost")
    ]
    trashed_titles = {post.get("title") for post in trashed_blogposts}
except Exception as e:
    api_error_3 = str(e)

assert_condition_3 = (
    api_error_3 is None and
    compare_is_list_subset([post_1, post_2], list(trashed_titles), list_comparison_function="all")
)
assert_message_3 = (
    f"Assertion 3 Failed: Error occurred while fetching trashed blog posts: {api_error_3}"
    if api_error_3 else
    f"Assertion 3 Failed: Expected both '{post_1}' and '{post_2}' to be present in the trashed state in space '{space}', "
    f"but one or both were missing."
)
assert assert_condition_3, assert_message_3


# Action

In [25]:
# proto_ignore
import confluence

**Simulated User**: Can you restore some blog posts for me?

**Action Agent**: Of course. I can help with that. Do you have the titles of the blog posts you would like to restore?

**Simulated User**: "Annual Tech Summit Recap"  and "Q3 Marketing Goals".

In [26]:
confluence.get_spaces()

[{'spaceKey': 'DOC',
  'name': 'Documentation Space',
  'description': 'All official documentation'},
 {'spaceKey': 'BLOG',
  'name': 'Blog Space',
  'description': 'ECorporate blog posts'},
 {'spaceKey': 'MARKETING',
  'name': 'Marketing',
  'description': 'Marketing strategies and campaigns'}]

In [27]:
confluence.get_content_list(spaceKey="BLOG", status="trashed")

[{'id': '4',
  'type': 'blogpost',
  'spaceKey': 'BLOG',
  'title': 'Weekly Update',
  'status': 'trashed',
  'body': {'storage': '<p>This post is trashed.</p>'},
  'postingDay': '2024-01-10'},
 {'id': '6',
  'type': 'blogpost',
  'spaceKey': 'BLOG',
  'title': 'Annual Tech Summit Recap',
  'status': 'trashed',
  'body': {'storage': {'value': 'The Annual Tech Summit featured discussions on AI, quantum computing, and next-gen networking. Keynote sessions highlighted industry shifts and future strategies.',
    'representation': 'storage'}},
  'postingDay': None,
  'link': '/content/6'},
 {'id': '7',
  'type': 'blogpost',
  'spaceKey': 'BLOG',
  'title': 'Q3 Marketing Goals',
  'status': 'trashed',
  'body': {'storage': {'value': 'Our Q3 marketing goals include expanding outreach in Europe, boosting engagement through webinars, and launching a new social media campaign for product awareness.',
    'representation': 'storage'}},
  'postingDay': None,
  'link': '/content/7'}]

In [28]:
confluence.update_content(id='6', body={'status': 'current'})

{'id': '6',
 'type': 'blogpost',
 'spaceKey': 'BLOG',
 'title': 'Annual Tech Summit Recap',
 'status': 'current',
 'body': {'storage': {'value': 'The Annual Tech Summit featured discussions on AI, quantum computing, and next-gen networking. Keynote sessions highlighted industry shifts and future strategies.',
   'representation': 'storage'}},
 'postingDay': None,
 'link': '/content/6',
 'version': {'number': 1, 'minorEdit': False}}

In [29]:
confluence.update_content(id='7', body={'status': 'current'})

{'id': '7',
 'type': 'blogpost',
 'spaceKey': 'BLOG',
 'title': 'Q3 Marketing Goals',
 'status': 'current',
 'body': {'storage': {'value': 'Our Q3 marketing goals include expanding outreach in Europe, boosting engagement through webinars, and launching a new social media campaign for product awareness.',
   'representation': 'storage'}},
 'postingDay': None,
 'link': '/content/7',
 'version': {'number': 1, 'minorEdit': False}}

**Action Agent**: I have restored the blog posts titled "Annual Tech Summit Recap" and "Q3 Marketing Goals".

# Final Assertion

1. Assert that the posts "Annual Tech Summit Recap" and "Q3 Marketing Goals" in the "BLOG" space do not have the status "trashed".

In [30]:
from Scripts.assertions_utils import *
import confluence

# --- Constants ---
space = "BLOG"
post_1 = "Annual Tech Summit Recap"
post_2 = "Q3 Marketing Goals"

# --- Assertion 1: Assert that both posts are restored and appear in the current state in the "BLOG" space ---
api_error_2 = None
current_titles = set()

try:
    # Fetch all current (active) blog posts from the "BLOG" space
    current_blogposts = confluence.get_content_list(spaceKey=space, status="current")
    current_blogposts = [
        post for post in current_blogposts
        if compare_strings(post.get("type"), "blogpost")
    ]
    current_titles = {post.get("title") for post in current_blogposts}
except Exception as e:
    api_error_2 = str(e)

# --- Assertion condition (simplified) ---
# We only verify that both posts exist in the current state
assert_condition_1 = (
    api_error_2 is None and
    compare_is_list_subset([post_1, post_2], list(current_titles), list_comparison_function="all")
)

# --- Assertion message ---
assert_message_1 = (
    f"Assertion 1 Failed: Error occurred while fetching content: {api_error_2}"
    if api_error_2
    else f"Assertion 1 Failed: Expected blog posts '{post_1}' and '{post_2}' to be present in the current state, "
         f"but missing from current: {[title for title in [post_1, post_2] if title not in current_titles]}"
)

# --- Final check ---
assert assert_condition_1, assert_message_1
